In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import(
    when, col, substring, expr, 
    to_date, year, sum, count, lit,
    regexp_replace, lpad, length,
    month, count
)
spark = SparkSession.builder.appName("agrupar-sumarizar") \
    .config("spark.sql.legacy.timeParserPolicy","LEGACY") \
    .config("spark.sql.repl.eagerEval.enabled", "true") \
    .config("spark.sql.repl.eagerEval.maxNumRows", 20) \
    .getOrCreate()
df = spark.read.csv("curso-analise-de-dados/dados/sim_salvador_2023.csv",
                    header=True,
                    inferSchema=True
)
df.show(5)

+---------+--------+-----+----+-------+--------+--------+
|CODMUNRES| DTOBITO|IDADE|SEXO|RACACOR|CAUSABAS|TIPOBITO|
+---------+--------+-----+----+-------+--------+--------+
|   292720| 2012023|  457|   2|      4|    J980|       2|
|   292720| 5012023|  482|   2|      2|    I500|       2|
|   292720| 7012023|  452|   2|      4|    C220|       2|
|   292720| 8012023|  456|   2|      4|    W789|       2|
|   292720|10012023|  468|   1|      4|    X954|       2|
+---------+--------+-----+----+-------+--------+--------+
only showing top 5 rows



In [15]:
# Resumo funcional Esse código:
# Corrige o formato da data de óbito (DTOBITO).
# Converte o campo para um tipo de data válido.
# Cria uma coluna derivada com o ano do óbito, facilitando análises temporais.
# DTOBITO com date e ano_obito
df = df.withColumn(
    "DTOBITO_clean", 
    #Converte a coluna DTOBITO para string.
    # Garante que ela tenha exatamente 8 caracteres, preenchendo com zeros à esquerda quando necessário.
    lpad(col("DTOBITO").cast("string"), 8, "0")
).withColumn(
    # data no formato ddMMyyyy precisam ter 8 dígitos.
    # Exemplo:
    # 1012023 → 01012023
    # 1122022 → 01122022
    # conversão para o tipo date
    "DTOBITO_dt", 
    # Converte a coluna DTOBITO_clean de string para o tipo Date do Spark.
    # Usa explicitamente o formato ddMMyyyy.
    # DTOBITO_dt passa a ser uma data válida, permitindo operações temporais (filtros, agrupamentos, etc.).
    to_date(col("DTOBITO_clean"), "ddMMyyyy")
).withColumn(
    # Extrai o ano da data de óbito.
    # Cria a coluna ano_obito como um inteiro.
    # Exemplo:
    # 2023-01-01 → 2023
    "ano_obito", 
    year(col("DTOBITO_dt"))
)
df_obitos_por_mes = df \
    .groupBy(year("DTOBITO_dt").alias("ano"),
            month("DTOBITO_dt").alias("mes")) \
    .agg(count("*").alias("obitos_totais")) \
    .orderBy("ano","mes")
df_obitos_por_mes.show()

+----+---+-------------+
| ano|mes|obitos_totais|
+----+---+-------------+
|2023|  1|           23|
|2023|  2|           15|
|2023|  3|           14|
|2023|  4|           22|
|2023|  5|           30|
|2023|  6|           25|
|2023|  7|           20|
|2023|  8|           20|
|2023|  9|           22|
|2023| 10|           20|
|2023| 11|           19|
|2023| 12|           31|
+----+---+-------------+

